# Lab 5: Central Gateway を使用した Supervisor エージェント

## 概要
Central Gateway を通じて3つの専門エージェントをオーケストレーションする、AgentCore Runtime にデプロイされた Supervisor エージェントを構築します:
- **Diagnostics Agent**（Lab 02 からの Lambda）
- **Remediation Agent**（Lab 03a からの Runtime）
- **Prevention Agent**（Lab 04 からの Runtime）

## アーキテクチャ
```
┌─────────────────────────────────────────────────────────────────────┐
│                              ユーザー                                │
│                    （Cognito JWT トークン）                          │
└────────────────────────────┬────────────────────────────────────────┘
                             │
                             │ HTTP POST /invocations
                             │ Authorization: Bearer <JWT>
                             ↓
┌─────────────────────────────────────────────────────────────────────┐
│                    SUPERVISOR エージェント（ローカル）                │
│                                                                     │
│  • インシデント対応ワークフローをオーケストレーション                   │
│  • MCP 経由で3つの専門エージェントを調整                              │
└────────────────────────────┬────────────────────────────────────────┘
                             │
                             │ JWT 付き MCP（Authorization ヘッダーを渡す）
                             ↓
┌─────────────────────────────────────────────────────────────────────┐
│                    CENTRAL GATEWAY（MCP）                           │
│  • すべてのエージェント通信用の単一 Gateway                          │
│  • JWT 認可（Cognito）                                              │
│  • REQUEST フェーズでの Lambda インターセプター                       │
└────────────────────────────┬────────────────────────────────────────┘
                             │
                             │ インターセプターが JWT を検証
                             ↓
┌─────────────────────────────────────────────────────────────────────┐
│                    LAMBDA インターセプター（Lab 03b）                │
│  • JWT から cognito:groups を抽出                                   │
│  • Approvers: すべてのツールにフルアクセス                           │
│  • SRE: 制限付きアクセス（診断 + 予防のみ）                          │
└────────────────────────────┬────────────────────────────────────────┘
                             │
                    ┌────────┴────────┐
                    ↓                 ↓
                 許可              拒否
                    ↓                 ↓
         ターゲットにルーティング   エラーを返す
                    │
    ┌───────────────┼───────────────┬───────────────┐
    │               │               │               │
    ↓               ↓               ↓               ↓
┌─────────┐   ┌─────────┐   ┌─────────┐   ┌─────────┐
│ ターゲット1│   │ ターゲット2│   │ ターゲット3│   │（将来）  │
│         │   │         │   │         │   │         │
│ Lambda  │   │ Runtime │   │ Runtime │   │ エージェント │
│(Lab 02) │   │(Lab 03a)│   │ (Lab 04)│   │         │
│         │   │         │   │         │   │         │
│ 診断    │   │ 修復    │   │ 予防    │   │   ...   │
└─────────┘   └─────────┘   └─────────┘   └─────────┘
     │             │             │
     │ IAM ロール   │ OAuth2 M2M  │ OAuth2 M2M
     │             │             │
     ↓             ↓             ↓
┌─────────────────────────────────────────────────────────────────────┐
│                    AWS インフラストラクチャ                          │
│  • CloudWatch Logs & Metrics                                        │
│  • DynamoDB テーブル                                                 │
│  • EC2 インスタンス                                                  │
│  • AgentCore Code Interpreter                                       │
│  • AgentCore Browser                                                │
└─────────────────────────────────────────────────────────────────────┘
```

### 主要コンポーネント

1. **Supervisor エージェント（Runtime）**
   - JWT 付きの `/invocations` への HTTP POST を受信
   - Authorization ヘッダーを抽出して MCP クライアントに渡す
   - すべての専門エージェントをオーケストレーション

2. **Central Gateway**
   - すべてのエージェント通信用の単一 Gateway
   - Cognito 経由の JWT 認証
   - Lambda インターセプターがリクエストを認可

3. **Lambda インターセプター（Lab 03b）**
   - Lab 03b から再利用
   - JWT を検証し cognito:groups をチェック
   - ロールベースの権限を強制

4. **3つの専門エージェント**
   - **Diagnostics**（Lambda）: ログとメトリクスを分析
   - **Remediation**（Runtime）: Code Interpreter で修正を実行
   - **Prevention**（Runtime）: Browser でベストプラクティスを調査

### 認証フロー

```
ユーザー → Cognito（JWT）
         ↓
HTTP POST /invocations（Authorization: Bearer <JWT>）
         ↓
Supervisor が Authorization ヘッダーを抽出
         ↓
MCP クライアント → Central Gateway（Authorization ヘッダー付き）
         ↓
インターセプターが JWT と cognito:groups を検証
         ↓
┌────────┴────────┐
↓                 ↓
Approvers         SRE
（すべてのツール）  （診断 + 予防のみ）
```

## 前提条件
- ✅ Lab 02 完了（Diagnostics Lambda）
- ✅ Lab 03a 完了（Remediation Runtime）
- ✅ Lab 03b 完了（インターセプター Lambda）
- ✅ Lab 04 完了（Prevention Runtime）

## 0. 依存関係のインストール

In [ ]:
%pip install -q -r requirements.txt
print("✅ 依存関係がインストールされました")

## 1. インポートと設定

In [ ]:
import json
import boto3
import time
import shutil
import urllib.parse

from bedrock_agentcore_starter_toolkit import Runtime
from lab_helpers.config import MODEL_ID, AWS_REGION, WORKSHOP_NAME
from lab_helpers.parameter_store import get_parameter, put_parameter
from lab_helpers.constants import PARAMETER_PATHS
from lab_helpers.lab_01.infrastructure import get_app_url

print(f"✅ インポート完了")
print(f"   リージョン: {AWS_REGION}")
print(f"   ワークショップ: {WORKSHOP_NAME}")

## 2. 前提条件の確認

In [ ]:
# 必要なすべてのリソースが存在するか確認
try:
    lab02_lambda_arn = get_parameter(PARAMETER_PATHS['lab_02']['lambda_function_arn'])
    lab03_runtime_arn = get_parameter(PARAMETER_PATHS['lab_03']['runtime_arn'])
    lab04_runtime_arn = get_parameter(PARAMETER_PATHS['lab_04']['runtime_arn'])
    interceptor_arn = get_parameter(PARAMETER_PATHS['lab_03b']['interceptor_function_arn'])
    user_pool_id = get_parameter(PARAMETER_PATHS['cognito']['user_pool_id'])
    
    print("✅ すべての前提条件を確認しました")
    print(f"   Lab 02 Lambda: {lab02_lambda_arn}")
    print(f"   Lab 03a Runtime: {lab03_runtime_arn}")
    print(f"   Lab 04 Runtime: {lab04_runtime_arn}")
    print(f"   インターセプター: {interceptor_arn}")
    print(f"   Cognito Pool: {user_pool_id}")
except Exception as e:
    print(f"❌ 前提条件が不足しています: {e}")
    raise

## 3. インターセプター付きの Central Gateway を作成

In [ ]:
# Gateway サービスロールを作成
iam = boto3.client('iam', region_name=AWS_REGION)
sts = boto3.client('sts', region_name=AWS_REGION)
account_id = sts.get_caller_identity()['Account']

gateway_role_name = f"{WORKSHOP_NAME}_CentralGatewayRole"
gateway_trust = {
    "Version": "2012-10-17",
    "Statement": [{"Effect": "Allow", "Principal": {"Service": "bedrock-agentcore.amazonaws.com"}, "Action": "sts:AssumeRole"}]
}
gateway_permissions = {
    "Version": "2012-10-17",
    "Statement": [
        {"Effect": "Allow", "Action": "lambda:InvokeFunction", "Resource": "*"},
        {"Effect": "Allow", "Action": "bedrock-agentcore:*", "Resource": "*"},
        {"Effect": "Allow", "Action": "logs:*", "Resource": "*"},
        {"Effect": "Allow",
                "Action": [
                    "bedrock-agentcore:*",
                    "bedrock:*",
                    "agent-credential-provider:*",
                    "iam:PassRole",
                    "secretsmanager:GetSecretValue",
                    "lambda:InvokeFunction"
                ],
                "Resource": "*"
            }            
    ]
}

try:
    gw_role = iam.create_role(RoleName=gateway_role_name, AssumeRolePolicyDocument=json.dumps(gateway_trust))
    gateway_role_arn = gw_role['Role']['Arn']
    time.sleep(10)
except iam.exceptions.EntityAlreadyExistsException:
    gateway_role_arn = iam.get_role(RoleName=gateway_role_name)['Role']['Arn']

iam.put_role_policy(RoleName=gateway_role_name, PolicyName='GatewayPermissions', PolicyDocument=json.dumps(gateway_permissions))
print(f"✅ Gateway ロール: {gateway_role_arn}")

In [ ]:
# Gateway ロール ARN を Parameter Store に保存
put_parameter(
    PARAMETER_PATHS['lab_05']['gateway_role_arn'],
    gateway_role_arn,
    region_name=AWS_REGION
)

**JWT 認可付き Central Gateway の作成**

以下を持つ AgentCore Gateway を作成:
- カスタム JWT Authorizer（Cognito Discovery URL）
- 許可クライアント: User Auth + M2M
- リクエスト認可用の Lambda インターセプター
- Gateway ID と URL を Parameter Store に保存

In [ ]:
# Cognito 設定を取得
user_auth_client_id = get_parameter(PARAMETER_PATHS['cognito']['user_auth_client_id'])
m2m_client_id = get_parameter(PARAMETER_PATHS['cognito']['m2m_client_id'])
discovery_url = f"https://cognito-idp.{AWS_REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration"

# インターセプター付きの Gateway を作成
agentcore = boto3.client('bedrock-agentcore-control', region_name=AWS_REGION)
gateway = agentcore.create_gateway(
    name="aiml301-central-gateway",
    roleArn=gateway_role_arn,
    protocolType="MCP",
    authorizerType="CUSTOM_JWT",
    authorizerConfiguration={"customJWTAuthorizer": {"discoveryUrl": discovery_url, "allowedClients": [user_auth_client_id, m2m_client_id]}},
    interceptorConfigurations=[{
        "interceptionPoints": ["REQUEST"],
        "interceptor": {"lambda": {"arn": interceptor_arn}},
        "inputConfiguration": {"passRequestHeaders": True}
    }]
)

central_gateway_id = gateway['gatewayId']
central_gateway_url = gateway['gatewayUrl']
put_parameter(
    PARAMETER_PATHS['lab_05']['gateway_id'],
    central_gateway_id,
    region_name=AWS_REGION
)

put_parameter(
    PARAMETER_PATHS['lab_05']['gateway_url'],
    central_gateway_url,
    region_name=AWS_REGION
)

print(f"✅ Gateway を作成しました: {central_gateway_id}")
print(f"   URL: {central_gateway_url}")

In [ ]:
!aws sts get-caller-identity


In [ ]:
# Lambda 権限を追加
lambda_client = boto3.client('lambda', region_name=AWS_REGION)
try:
    lambda_client.add_permission(
        FunctionName=interceptor_arn.split(':')[-1],
        StatementId=f'AllowGateway{central_gateway_id[:8]}',
        Action='lambda:InvokeFunction',
        Principal='bedrock-agentcore.amazonaws.com',
        SourceArn=f'arn:aws:bedrock-agentcore:{AWS_REGION}:{account_id}:gateway/{central_gateway_id}'
    )
    print("✅ Lambda 権限を追加しました")
except lambda_client.exceptions.ResourceConflictException:
    print("ℹ️  権限は既に存在します")

## 4. ターゲット 1 の追加: Diagnostics Lambda

In [ ]:
diagnostics_schema = [{
    "name": "logs_analysis_agent",
    "description": "This agent is a system diagnostics tool that analyzes AWS infrastructure health by examining logs and metrics from EC2, NGINX, and DynamoDB services. It uses AI to correlate findings across different data sources, identify issues like throttling, high resource utilization, and application errors, then provides evidence-based assessments with recommended actions for troubleshooting system problems.",
    "inputSchema": {"type": "object", "properties": {"query": {"type": "string"}}, "required": ["query"]}
}]

target1 = agentcore.create_gateway_target(
    gatewayIdentifier=central_gateway_id,
    name="strands-diagnostics-agent",
    targetConfiguration={"mcp": {"lambda": {"lambdaArn": lab02_lambda_arn, "toolSchema": {"inlinePayload": diagnostics_schema}}}},
    credentialProviderConfigurations=[{"credentialProviderType": "GATEWAY_IAM_ROLE"}]
)
print(f"✅ Target 1: {target1['targetId']}")

## 5. ターゲット 2 の追加: Remediation Runtime

In [ ]:
# OAuth2 プロバイダーを取得
try:
    oauth2_provider_arn = get_parameter(PARAMETER_PATHS['lab_03']['oauth2_provider_arn'])
except:
    m2m_client_secret = get_parameter(PARAMETER_PATHS['cognito']['m2m_client_secret'])
    cred = agentcore.create_oauth2_credential_provider(
        name='aiml301-m2m-lab05a',
        credentialProviderVendor='CustomOauth2',
        oauth2ProviderConfigInput={'customOauth2ProviderConfig': {'clientId': m2m_client_id, 'clientSecret': m2m_client_secret, 'oauthDiscovery': {'discoveryUrl': discovery_url}}}
    )
    oauth2_provider_arn = cred['credentialProviderArn']

resource_server_id = get_parameter(PARAMETER_PATHS['cognito']['resource_server_identifier'])
m2m_scopes = [f"{resource_server_id}/mcp.invoke", f"{resource_server_id}/runtime.access"]

encoded_arn = urllib.parse.quote(lab03_runtime_arn, safe='')
remediation_endpoint = f"https://bedrock-agentcore.{AWS_REGION}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

target2 = agentcore.create_gateway_target(
    gatewayIdentifier=central_gateway_id,
    name="aiml301-runtime-target",
    targetConfiguration={"mcp": {"mcpServer": {"endpoint": remediation_endpoint}}},
    credentialProviderConfigurations=[{"credentialProviderType": "OAUTH", "credentialProvider": {"oauthCredentialProvider": {"providerArn": oauth2_provider_arn, "scopes": m2m_scopes}}}]
)
print(f"✅ ターゲット 2: {target2['targetId']}")

## 6. ターゲット 3 の追加: Prevention Runtime

In [ ]:
encoded_arn = urllib.parse.quote(lab04_runtime_arn, safe='')
prevention_endpoint = f"https://bedrock-agentcore.{AWS_REGION}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

target3 = agentcore.create_gateway_target(
    gatewayIdentifier=central_gateway_id,
    name="aiml301-prevention-runtime-target",
    targetConfiguration={"mcp": {"mcpServer": {"endpoint": prevention_endpoint}}},
    credentialProviderConfigurations=[{"credentialProviderType": "OAUTH", "credentialProvider": {"oauthCredentialProvider": {"providerArn": oauth2_provider_arn, "scopes": m2m_scopes}}}]
)
print(f"✅ ターゲット 3: {target3['targetId']}")

**Gateway ターゲットの同期**

ターゲットの同期を開始し、Gateway のステータスをポーリング:
- `AVAILABLE`: 同期成功
- `FAILED`: 理由付きで同期失敗
- 30回試行後（60秒）タイムアウト

In [ ]:
time.sleep(20)

In [ ]:
# Runtime ターゲットのみを同期（Lambda はスキップ）
for target in [target2, target3]:  # Remediation と Prevention（Runtime）
    agentcore.synchronize_gateway_targets(
        gatewayIdentifier=central_gateway_id,
        targetIdList=[target['targetId']]
    )
    print(f"⏳ 同期中: {target['targetId']}...")

# 同期ステータスをポーリング
import time
max_attempts = 30
for i in range(max_attempts):
    time.sleep(2)
    response = agentcore.get_gateway(gatewayIdentifier=central_gateway_id)
    status = response['status']
    
    if status == 'READY':
        print("✅ Runtime ターゲットの同期が成功しました")
        break
    elif status == 'FAILED':
        print(f"❌ 同期に失敗しました: {response.get('failureReasons', [])}")
        break
    else:
        print(f"   ステータス: {status}（{i+1}/{max_attempts}）")
else:
    print("⚠️ 同期タイムアウト - Gateway のステータスを手動で確認してください")

## 7. トークンの取得

In [ ]:
# Approver として認証
cognito = boto3.client('cognito-idp', region_name=AWS_REGION)
approver_username = get_parameter(PARAMETER_PATHS['cognito']['approver_user_email'])
approver_password = get_parameter(PARAMETER_PATHS['cognito']['approver_user_password'])

auth_response = cognito.initiate_auth(
    ClientId=user_auth_client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={'USERNAME': approver_username, 'PASSWORD': approver_password}
)
access_token = auth_response['AuthenticationResult']['AccessToken']
print("✅ Approver として認証しました")

In [ ]:
print(access_token)

## 8. Supervisor コードの実行

**ローカル Supervisor エージェントの実行**

Strands フレームワークを使用して Supervisor エージェントをローカルで実行:
- JWT 認証で Central Gateway に接続
- Gateway ターゲットから利用可能なすべてのツールを取得
- 診断、修復、予防エージェントをオーケストレーション
- ユーザークエリを処理し、包括的なレスポンスを返す

### Supervisor ツールリスト呼び出し - Central Gateway に接続（3つのエージェントがツールとして公開）

In [ ]:
from lab_helpers.lab_05.local_supervisor_agent import run_supervisor_agent

# クエリを設定
user_prompt = "利用可能なすべてのツールをリストしてください"

# Supervisor エージェントを実行
response = run_supervisor_agent(
    gateway_url=central_gateway_url,
    access_token=access_token,
    prompt=user_prompt
)

print("\n" + "="*80)
print("SUPERVISOR エージェントのレスポンス")
print("="*80)
print(response)

### Diagnostic エージェント呼び出し

In [ ]:
# クエリを設定
user_prompt = "CRM アプリケーションで DynamoDB スロットリングに関連する最近の問題は何ですか"

# Supervisor エージェントを実行
response = run_supervisor_agent(
    gateway_url=central_gateway_url,
    access_token=access_token,
    prompt=user_prompt
)

print("\n" + "="*80)
print("SUPERVISOR エージェントのレスポンス")
print("="*80)
print(response)

**Streamlit 用のクエリ例**

Supervisor エージェントで以下のクエリ例を試してください:

```python
# 診断クエリ
"CRM アプリケーションの DynamoDB スロットリングに関連する最近の問題は何ですか？"

# インフラストラクチャクエリ
"すべての DynamoDB テーブルをリストしてください。infrastructure エージェントと action_type=only_execute を使用"

# 予防クエリ
"テーブルスロットリングを回避するための DynamoDB ベストプラクティスを調査"

# 完全ワークフロー
"DynamoDB アクセス問題を解決するために IAM 権限を修正。必要な IAM 権限を持っており、常に action_type='only_execute' を使用"
```

## 9. Streamlit によるインタラクティブチャットインターフェース

ユーザーフレンドリーな UI で Supervisor エージェントと対話するための Web ベースのチャットインターフェースを起動します。

### 9.1 Gateway 設定ファイルの作成

In [ ]:
# Streamlit アプリ用の gateway_config.json を作成
# 注意: GatewayClient は Cognito 認証用に特定のフィールドを必要とします
gateway_config = {
    "gateway_id": central_gateway_id,
    "gateway_url": central_gateway_url,
    "region": AWS_REGION,
    "client_info": {
        "user_pool_id": user_pool_id,
        "client_id": user_auth_client_id,
        "client_secret": "",  # User Auth クライアントは空（シークレットなし）
        "scope": "openid",  # Cognito に必要なスコープ
        "username": approver_username,
        "password": approver_password
    }
}

import json
with open("gateway_config.json", "w") as f:
    json.dump(gateway_config, f, indent=2)

print("✅ Gateway 設定を gateway_config.json に保存しました")
print(f"   Gateway ID: {central_gateway_id}")
print(f"   Gateway URL: {central_gateway_url}")
print(f"   ユーザー: {approver_username}")

### 9.2 Streamlit のインストール

In [ ]:
%pip install -q streamlit
print("✅ Streamlit がインストールされました")

### 9.3 Streamlit アプリの起動

In [ ]:
import subprocess
import time
import os
import json
import boto3

def get_streamlit_url():
    """
    SageMaker Studio またはローカル環境用のアクセス可能な Streamlit URL を生成
    """
    try:
        # SageMaker Studio で実行中かチェック
        with open("/opt/ml/metadata/resource-metadata.json", "r") as file:
            data = json.load(file)
            domain_id = data["DomainId"]
            space_name = data["SpaceName"]
        
        # SageMaker Studio URL を取得
        sagemaker_client = boto3.client("sagemaker")
        response = sagemaker_client.describe_space(
            DomainId=domain_id, 
            SpaceName=space_name
        )
        streamlit_url = response["Url"] + "/proxy/8501/"
        print(f"📍 SageMaker Studio で実行中")
        print(f"   Domain ID: {domain_id}")
        print(f"   Space Name: {space_name}")
        
    except (FileNotFoundError, json.JSONDecodeError, KeyError):
        # ローカルまたは SageMaker Studio 外で実行中
        streamlit_url = "http://localhost:8501"
        print(f"📍 ローカル環境で実行中")
    
    return streamlit_url

# 既存の streamlit プロセスを終了
try:
    subprocess.run(["pkill", "-f", "streamlit"], check=False)
    time.sleep(2)
except:
    pass

# バックグラウンドで streamlit を起動
streamlit_process = subprocess.Popen(
    ["streamlit", "run", "lab_helpers/lab_05/streamlit_app.py", "--server.port", "8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    cwd=os.getcwd()
)

time.sleep(5)

# アクセス可能な URL を取得
streamlit_url = get_streamlit_url()

print("✅ Streamlit アプリを起動しました！")
print("\n" + "="*80)
print(f"🌐 チャットインターフェースにアクセス:\n{streamlit_url}")
print("="*80)
print("\n💡 ヒント:")
print("   • アプリはノートブックと同じ Gateway と認証を使用します")
print("   • 例: 'CRM アプリケーションでどのような問題が見られますか？' などのクエリを試してください")
print("   • サイドバーの 'Clear Chat History' をクリックして会話をリセット")
print("\n⚠️  アプリを停止するには、以下のクリーンアップセルを実行してください")

### 9.4 アーキテクチャ: Streamlit 統合

```
┌─────────────────────────────────────────────────────────────────┐
│                    ユーザー（Web ブラウザ）                       │
│                    http://localhost:8501                        │
└────────────────────────────┬────────────────────────────────────┘
                             │
                             │ HTTP
                             ↓
┌─────────────────────────────────────────────────────────────────┐
│                    STREAMLIT アプリ                             │
│  • ストリーミング付きチャットインターフェース                      │
│  • gateway_config.json を読み込み                               │
│  • Cognito で認証                                               │
│  • JWT 付きで MCP クライアントを作成                             │
└────────────────────────────┬────────────────────────────────────┘
                             │
                             │ MCP + JWT
                             ↓
┌─────────────────────────────────────────────────────────────────┐
│                    CENTRAL GATEWAY                              │
│  • インターセプター経由で JWT を検証                             │
│  • 適切なエージェントにルーティング                              │
└────────────────────────────┬────────────────────────────────────┘
                             │
                    ┌────────┴────────┬────────────────┐
                    ↓                 ↓                ↓
              Diagnostics       Remediation      Prevention
              （Lambda）         （Runtime）       （Runtime）
```

**主要機能:**

- **リアルタイムストリーミング**: エージェントのレスポンスを生成されるままに表示
- **チャット履歴**: 会話コンテキストを維持
- **ツールの可視性**: サイドバーで呼び出されているツールを表示
- **エラー処理**: 再試行の提案付きの適切なエラーメッセージ
- **OAuth 統合**: Cognito 経由の自動トークン管理

### 9.5 Streamlit アプリの停止

In [ ]:
# streamlit プロセスを停止
try:
    streamlit_process.terminate()
    streamlit_process.wait(timeout=5)
    print("✅ Streamlit アプリを停止しました")
except:
    subprocess.run(["pkill", "-f", "streamlit"], check=False)
    print("✅ Streamlit プロセスを終了しました")

In [ ]:
# ポート 80 と 8080 の両方で URL を試行
print(f"CRM アプリ UI にアクセスするにはこちらをクリック: '{get_app_url()}'")

## 10. クリーンアップ

In [ ]:
# Lab 05a リソースのみをクリーンアップ
print("🗑️  クリーンアップ中...")

# 'aiml301-central-gateway' を含む ID を持つ Gateway を検索
central_gateway_id = None
try:
    gateways = agentcore.list_gateways().get('items', [])
    for gateway in gateways:
        if 'aiml301-central-gateway' in gateway.get('gatewayId', ''):
            central_gateway_id = gateway['gatewayId']
            print(f"Central Gateway を発見: {central_gateway_id}")
            break
except Exception as e:
    print(f"Gateway の検索エラー: {e}")

if central_gateway_id:
    # まずすべてのターゲットを削除
    try:
        targets = agentcore.list_gateway_targets(gatewayIdentifier=central_gateway_id).get('items', [])
        print(f"削除する {len(targets)} 個のターゲットを発見")
        for target in targets:
            target_id = target['targetId']
            print(f"ターゲットを削除中: {target_id}")
            agentcore.delete_gateway_target(gatewayIdentifier=central_gateway_id, targetId=target_id)
            time.sleep(2)
        print("✅ すべてのターゲットを削除しました")
    except Exception as e:
        print(f"ターゲットの削除エラー: {e}")

    # ターゲットが完全に削除されるまで待機
    time.sleep(5)
    
    # Gateway を削除
    try:
        print(f"Gateway を削除中: {central_gateway_id}")
        response = agentcore.delete_gateway(gatewayIdentifier=central_gateway_id)
        print(f"Gateway 削除レスポンス: {response}")
        time.sleep(3)
        print("✅ Gateway を削除しました")
    except Exception as e:
        print(f"❌ Gateway の削除エラー: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️  'aiml301-central-gateway' を含む ID を持つ Gateway が見つかりません")

print("✅ クリーンアップ完了")